In [76]:
import pandas as pd
import numpy as np
import re
import string
import pickle

# Load Model

In [77]:
with open('../static/model/model.pickle', 'rb') as file:
    model = pickle.load(file)

# Load vectorizer

In [78]:
with open('../static/model/vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)

# Text Preprocessing

In [79]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [80]:

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [81]:
def preprocessing(text):
    # Convert to lowercase and split
    text = text.lower()
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove punctuation and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in sw]
    return " ".join(words)

# Get Prediction

In [82]:
# Test with different sentiments
test_texts = ["good", "bad", "terrible", "excellent", "hate this"]
for txt in test_texts:
    preprocessed_txt = preprocessing(txt)
    vectorized_txt = vectorizer.transform([preprocessed_txt])
    prediction = model.predict(vectorized_txt)
    print(f"Text: '{txt}' -> Prediction: {prediction[0]}")

Text: 'good' -> Prediction: 0
Text: 'bad' -> Prediction: 1
Text: 'terrible' -> Prediction: 1
Text: 'excellent' -> Prediction: 0
Text: 'hate this' -> Prediction: 1
